In [ ]:
from ultralytics import YOLO
# 첫번째 학습
# ✅ YOLOv8 모델 로드 (pre-trained weights)
model = YOLO("/home/alpaco/lcm/yolov8n.pt")

# ✅ 학습 실행
model.train(
    data="/home/alpaco/lcm/data_lcm.yaml",
    epochs=50,
    batch=16,
    imgsz=640, #이미지 크기 640 
    lr0=0.001,
    freeze=[0,1,2,3,4,5],
    exist_ok=False          # 같은 이름이 있어도 덮어쓰지 않고 새로운 폴더 생성
)

# ✅ 학습 완료 후 모델 저장
model.export(format="pt")  # PyTorch 모델 저장
print("\n✅ 학습 완료! 최적의 모델이 저장되었습니다. 🚀")


In [ ]:
# 영상 테스트
from ultralytics import YOLO

# :흰색_확인_표시: 훈련된 모델 로드
model = YOLO('/home/alpaco/lcm/runs/train/experiment_2/weights/best.pt')
# :흰색_확인_표시: 영상 파일에서 객체 탐지 수행
model.predict(
    source="/home/alpaco/lcm/testdatas/1000003186.mp4",  # 탐지할 영상 파일
    save=True,
    # conf=0.5,
    # iou=0.45,  # IoU 임계값 (객체 중복 제거)
    device="cuda",  # GPU 사용 (속도 향상)
    half=True  # FP16 연산 (속도 향상)
)

In [ ]:
def fix_track_objects_in_video(model_path, video_path, output_path=None, conf_threshold=0.25):
    """
    비디오에서 객체를 탐지하고 추적하는 함수
    
    Args:
        model_path (str): YOLO 모델 경로
        video_path (str): 입력 비디오 파일 경로
        output_path (str, optional): 결과 저장 비디오 파일 경로
        conf_threshold (float): 신뢰도 임계값
    
    Returns:
        dict: 추적 결과 통계
    """
    import cv2
    import time
    import os
    from ultralytics import YOLO
    
    # 모델 로드
    model = YOLO(model_path)
    
    # 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)
    
    # 비디오 속성
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 출력 비디오 설정
    if output_path:
        # 출력 디렉토리 확인
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # 처리 통계
    stats = {
        "total_frames": 0,
        "detected_frames": 0,
        "detection_counts": {},
        "tracking_ids": set(),
        "processing_time": 0
    }
    
    start_time = time.time()
    
    # 트래킹 상태 유지를 위한 변수
    last_tracked_objs = {}  # 마지막으로 추적된 객체 상태 저장
    next_id = 1             # 추적 ID 시작 번호
    memory_frames = 30      # 가려짐 발생 시 객체 기억 프레임 수
    
    try:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
                
            stats["total_frames"] += 1
            
            # 진행 상황 표시
            if stats["total_frames"] % 30 == 0:
                elapsed = time.time() - start_time
                progress = stats["total_frames"] / total_frames * 100
                print(f"\r진행: {progress:.1f}% ({stats['total_frames']}/{total_frames}), "
                      f"FPS: {stats['total_frames']/elapsed:.1f}", end="")
            
            # 기본 추적 수행
            try:
                # 기본 파라미터만 사용하는 방식
                results = model.track(
                    source=frame,
                    persist=True,         # 이전 추적 정보 유지 (가장 중요한 옵션)
                    conf=conf_threshold,  # 탐지 신뢰도 임계값
                    verbose=False
                )
            except Exception as e:
                print(f"\n추적 시 오류 발생: {e}")
                # 오류 발생 시 기본 탐지로 대체
                results = model(frame, conf=conf_threshold)
            
            # 결과 처리
            annotated_frame = results[0].plot()
            
            # 탐지된 객체 통계
            detected = False
            current_objs = {}
            
            for r in results:
                boxes = r.boxes
                if len(boxes) > 0:
                    detected = True
                    
                    for box in boxes:
                        cls = int(box.cls[0])
                        cls_name = model.names[cls]
                        
                        # 클래스별 카운트
                        if cls_name not in stats["detection_counts"]:
                            stats["detection_counts"][cls_name] = 0
                        stats["detection_counts"][cls_name] += 1
                        
                        # 추적 ID 처리
                        if hasattr(box, 'id') and box.id is not None:
                            track_id = int(box.id[0])
                            stats["tracking_ids"].add(track_id)
                            
                            # 현재 프레임의 객체 상태 저장
                            xyxy = box.xyxy[0].cpu().numpy()
                            conf = float(box.conf[0])
                            current_objs[track_id] = {
                                "class": cls_name,
                                "box": xyxy,
                                "conf": conf,
                                "last_seen": stats["total_frames"]
                            }
            
            # 객체 메모리 업데이트
            for track_id, obj_info in last_tracked_objs.items():
                # 현재 프레임에 없는 객체들 중에서 기억 시간이 지나지 않은 것만 유지
                if track_id not in current_objs:
                    if stats["total_frames"] - obj_info["last_seen"] <= memory_frames:
                        current_objs[track_id] = obj_info
            
            # 현재 프레임의 객체로 메모리 업데이트
            last_tracked_objs = current_objs.copy()
            
            if detected:
                stats["detected_frames"] += 1
            
            # 출력 비디오에 쓰기
            if output_path:
                out.write(annotated_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        print("\n처리가 중단되었습니다.")
    finally:
        # 자원 해제
        cap.release()
        if output_path and 'out' in locals():
            out.release()
        cv2.destroyAllWindows()
        
        # 최종 통계
        stats["processing_time"] = time.time() - start_time
        
        print("\n\n===== 처리 결과 =====")
        print(f"총 처리 프레임: {stats['total_frames']}/{total_frames}")
        print(f"객체 탐지된 프레임: {stats['detected_frames']} ({stats['detected_frames']/stats['total_frames']*100:.1f}%)")
        print(f"추적된 고유 객체 수: {len(stats['tracking_ids'])}")
        print(f"클래스별 탐지 횟수: {stats['detection_counts']}")
        print(f"총 처리 시간: {stats['processing_time']:.2f}초")
        print(f"평균 FPS: {stats['total_frames']/stats['processing_time']:.2f}")
        
        if output_path:
            print(f"결과 저장 경로: {output_path}")
    
    return stats